In [29]:
from IPython.display import Image, display

from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.runnables.config import RunnableConfig
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore
from trustcall import create_extractor
from pydantic import BaseModel, Field

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.store.base import BaseStore

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

import uuid
from langgraph.store.memory import InMemoryStore

In [30]:
from typing import TypedDict, List

class UserProfile(TypedDict):
    """User profile schema with typed fields"""
    user_name: str  # The user's preferred name
    interests: List[str]  # A list of the user's interests

In [31]:
# TypedDict instance
user_profile: UserProfile = {
    "user_name": "Marcio",
    "interests": ["ai", "technology", "malta", "speak english"]
}
user_profile

{'user_name': 'Marcio',
 'interests': ['ai', 'technology', 'malta', 'speak english']}

In [32]:
# Initialize the in-memory store
in_memory_store = InMemoryStore()

# Namespace for the memory to save
user_id = "1"
namespace_for_memory = (user_id, "memory")

# Save a memory to namespace as key and value
key = "user_profile"
value = user_profile
in_memory_store.put(namespace_for_memory, key, value)

In [33]:
# Get the memory by namespace and key
profile = in_memory_store.get(namespace_for_memory, "user_profile")
profile.value

{'user_name': 'Marcio',
 'interests': ['ai', 'technology', 'malta', 'speak english']}

In [ ]:


model = ChatOpenAI(model="llama3.1:8b", api_key="ollama",  temperature=0.7,   base_url="http://localhost:11434/v1")


# Schema 
class UserProfile(BaseModel):
    """ Profile of a user """
    user_name: str = Field(description="The user's preferred name")
    user_location: str = Field(description="The user's location")
    interests: list = Field(description="A list of the user's interests")

# Create the extractor
trustcall_extractor = create_extractor(
    model,
    tools=[UserProfile],
    tool_choice="UserProfile", # Enforces use of the UserProfile tool
)

# Chatbot instruction
MODEL_SYSTEM_MESSAGE = """You are a helpful assistant with memory that provides information about the user. 
If you have memory for this user, use it to personalize your responses.
Here is the memory (it may be empty): {memory}"""

# Extraction instruction
TRUSTCALL_INSTRUCTION = """Create or update the memory (JSON doc) to incorporate information from the following conversation:"""

def call_model(state: MessagesState, config: RunnableConfig, store: BaseStore):

    """Load memory from the store and use it to personalize the chatbot's response."""
    
    # Get the user ID from the config
    user_id = config["configurable"]["user_id"]

    # Retrieve memory from the store
    namespace = ("memory", user_id)
    existing_memory = store.get(namespace, "user_memory")

    # Format the memories for the system prompt
    if existing_memory and existing_memory.value:
        memory_dict = existing_memory.value
        formatted_memory = (
            f"Name: {memory_dict.get('user_name', 'Unknown')}\n"
            f"Location: {memory_dict.get('user_location', 'Unknown')}\n"
            f"Interests: {', '.join(memory_dict.get('interests', []))}"      
        )
    else:
        formatted_memory = None

    # Format the memory in the system prompt
    system_msg = MODEL_SYSTEM_MESSAGE.format(memory=formatted_memory)

    # Respond using memory as well as the chat history
    response = model.invoke([SystemMessage(content=system_msg)]+state["messages"])

    return {"messages": response}

def write_memory(state: MessagesState, config: RunnableConfig, store: BaseStore):

    """Reflect on the chat history and save a memory to the store."""
    
    # Get the user ID from the config
    user_id = config["configurable"]["user_id"]

    # Retrieve existing memory from the store
    namespace = ("memory", user_id)
    existing_memory = store.get(namespace, "user_memory")
        
    # Get the profile as the value from the list, and convert it to a JSON doc
    existing_profile = {"UserProfile": existing_memory.value} if existing_memory else None
    print("existing_profile")
    print(existing_profile)
    print("state")
    print(state)
    # Invoke the extractor
    result = trustcall_extractor.invoke({"messages": [SystemMessage(content=TRUSTCALL_INSTRUCTION)]+state["messages"], "existing": existing_profile})
    print("result")
    print(result)    
    # Get the updated profile as a JSON object
    updated_profile = result["responses"][0].model_dump()

    # Save the updated profile
    key = "user_memory"
    store.put(namespace, key, updated_profile)

# Define the graph
builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_node("write_memory", write_memory)
builder.add_edge(START, "call_model")
builder.add_edge("call_model", "write_memory")
builder.add_edge("write_memory", END)

# Store for long-term (across-thread) memory
across_thread_memory = InMemoryStore()

# Checkpointer for short-term (within-thread) memory
within_thread_memory = MemorySaver()

# Compile the graph with the checkpointer fir and store
graph = builder.compile(checkpointer=within_thread_memory, store=across_thread_memory)

# View
#display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [27]:
# We supply a thread ID for short-term (within-thread) memory
# We supply a user ID for long-term (across-thread) memory 
config = {"configurable": {"thread_id": "1", "user_id": "1"}}

# User input 
input_messages = [HumanMessage(content="Hi, my name is Marcio. I live in Malta and i love AI")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    # Check if the 'messages' key exists and the list is not empty
    if "messages" in chunk and chunk["messages"]:
        # Now it's safe to access the last element
        last_message = chunk["messages"][-1]
        # Check if the last message has the pretty_print method (good practice)
        if hasattr(last_message, 'pretty_print') and callable(last_message.pretty_print):
             last_message.pretty_print()
        else:
             print(f"Last message: {last_message}") # Fallback if no pretty_print
    # else:
        # Optional: print something if messages are empty or missing for debugging
        print("Skipping chunk with no messages or empty messages list.")
        # print(f"Current chunk keys: {chunk.keys()}")

================================ Human Message =================================

Hi, my name is Marcio. I live in Malta and i love AI
Skipping chunk with no messages or empty messages list.
================================== Ai Message ==================================

Hi Marcio! It's great to meet you. Living in Malta sounds wonderful, and it's fantastic that you love AI. If there's anything specific you'd like to discuss or learn about AI, feel free to ask!
Skipping chunk with no messages or empty messages list.
existing_profile
None
state
{'messages': [HumanMessage(content='Hi, my name is Marcio. I live in Malta and i love AI', additional_kwargs={}, response_metadata={}, id='de4b6e89-1577-402d-9b08-5696b630dc44'), AIMessage(content="Hi Marcio! It's great to meet you. Living in Malta sounds wonderful, and it's fantastic that you love AI. If there's anything specific you'd like to discuss or learn about AI, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata=

In [28]:
# User input 
input_messages = [HumanMessage(content="I like to study AI and pratice english in my free time")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I like to study AI and pratice english in my free time
================================== Ai Message ==================================

That sounds like a great way to spend your free time, Marcio! Studying AI can be really exciting, and practicing English will definitely help you communicate even better. If you ever want to discuss AI topics or practice your English skills, I'm here to help. Do you have any particular AI areas you're interested in or specific topics you'd like to explore?
existing_profile
{'UserProfile': {'user_name': 'Marcio', 'user_location': 'Malta', 'interests': ['AI']}}
state
{'messages': [HumanMessage(content='Hi, my name is Marcio. I live in Malta and i love AI', additional_kwargs={}, response_metadata={}, id='de4b6e89-1577-402d-9b08-5696b630dc44'), AIMessage(content="Hi Marcio! It's great to meet you. Living in Malta sounds wonderful, and it's fantastic that you love AI. If ther